# UD2.2 — Transformación de datos (caso: Titanic)

Objetivo: construir un flujo típico de preparación de datos para ML:
- Separar columnas numéricas/categóricas
- Imputación de nulos
- One-Hot / Ordinal encoding
- Escalado y su impacto en modelos basados en distancia (KNN)
- Pipeline para evitar fugas de datos (data leakage)

Regla clave:
- `fit` (imputer/encoder/scaler) **solo con train**; después `transform` en test.

Dataset: se carga desde `titanic.csv` (local) cuando está disponible.

## Paso 0: Importar las librerías necesarias

Además de pandas y numpy, importamos las herramientas de **sklearn** para transformación.

In [2]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Herramientas de sklearn para TRANSFORMACIÓN
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # Escalado
from sklearn.preprocessing import LabelEncoder, OneHotEncoder   # Codificación
from sklearn.preprocessing import OrdinalEncoder                # Codificación ordinal
from sklearn.compose import ColumnTransformer                   # Transformar por columnas
from sklearn.pipeline import Pipeline                           # Encadenar transformaciones
from sklearn.impute import SimpleImputer                        # Rellenar valores faltantes

# Para dividir datos y evaluar
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente")

print(f"Versión de Python: {sys.version}")
print(f"Pandas versión: {pd.__version__}")
print("Entorno listo para Aprendizaje Automático.")

ModuleNotFoundError: No module named 'sklearn'

## Paso 1: Cargar y preparar el dataset

Cargamos el Titanic y hacemos una limpieza básica antes de transformar.

In [ ]:
# Cargar el dataset del Titanic (prioridad: archivo local)
from pathlib import Path

candidatos = [
    Path("titanic.csv"),
    Path("contenidos/UD2/titanic.csv"),
]

df = None
origen = None

for ruta in candidatos:
    if ruta.exists():
        df = pd.read_csv(ruta)
        origen = str(ruta)
        break

if df is None:
    url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
    df = pd.read_csv(url)
    origen = url

print(f"Dataset cargado correctamente ({origen})")
print(f"Tamaño: {df.shape[0]} filas x {df.shape[1]} columnas")
df.head()

**Qué deberías ver aquí**
- Un `df.head()` con columnas típicas del Titanic (`Survived`, `Pclass`, `Sex`, `Age`, ...).
- El tamaño del dataset impreso.
- Si la ruta es local, el origen indicará `titanic.csv`.

In [ ]:
# Limpieza básica: rellenar valores faltantes
# (Lo aprendimos en el capítulo anterior)

df_limpio = df.copy()

# Rellenar Age con la mediana
df_limpio['Age'] = df_limpio['Age'].fillna(df_limpio['Age'].median())

# Rellenar Embarked con la moda (valor más frecuente)
df_limpio['Embarked'] = df_limpio['Embarked'].fillna(df_limpio['Embarked'].mode()[0])

# Eliminar columnas que no usaremos (tienen demasiados valores únicos o faltantes)
df_limpio = df_limpio.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])

print("Valores faltantes después de limpieza:")
print(df_limpio.isnull().sum())
print(f"\nColumnas: {list(df_limpio.columns)}")

---

# EJERCICIO 1: Escalado de Variables Numéricas (Básico)

El escalado es crucial para algoritmos que calculan distancias (KNN, SVM) o usan gradiente (Redes Neuronales).

Vamos a:
1. Identificar las columnas numéricas
2. Aplicar StandardScaler (media=0, std=1)
3. Aplicar MinMaxScaler (valores entre 0 y 1)
4. Comparar los resultados visualmente

---

## 1.1 Identificar columnas numéricas

In [ ]:
# Identificar columnas numéricas y categóricas
columnas_numericas = df_limpio.select_dtypes(include=['int64', 'float64']).columns.tolist()
columnas_categoricas = df_limpio.select_dtypes(include=['object']).columns.tolist()

print("COLUMNAS NUMÉRICAS:")
print(columnas_numericas)

print("\nCOLUMNAS CATEGÓRICAS:")
print(columnas_categoricas)

In [ ]:
# Ver las estadísticas de las columnas numéricas ANTES de escalar
print("ESTADÍSTICAS ANTES DEL ESCALADO:")
print("="*60)
df_limpio[columnas_numericas].describe()

### Problema: Diferentes escalas

Observa:
- **Age**: valores entre 0.42 y 80 años
- **Fare**: valores entre 0 y 512 libras
- **Pclass**: valores 1, 2, 3
- **SibSp/Parch**: valores pequeños (0-8)

Si usamos KNN sin escalar, el algoritmo pensará que `Fare` es mucho más importante que `Pclass` porque tiene valores más grandes.

## 1.2 Aplicar StandardScaler

**StandardScaler** transforma los datos para que tengan:
- Media = 0
- Desviación estándar = 1

Fórmula: $z = \frac{x - \mu}{\sigma}$

In [ ]:
# Seleccionar solo columnas numéricas para escalar
# Excluimos 'Survived' porque es la variable objetivo (y)
cols_a_escalar = ['Age', 'Fare', 'SibSp', 'Parch']

# Crear el escalador
scaler_standard = StandardScaler()

# fit_transform: 
#   1. fit() calcula la media y desviación de cada columna
#   2. transform() aplica la transformación
datos_escalados_standard = scaler_standard.fit_transform(df_limpio[cols_a_escalar])

# Convertir a DataFrame para ver mejor
df_standard = pd.DataFrame(
    datos_escalados_standard, 
    columns=[col + '_scaled' for col in cols_a_escalar]
)

print("DATOS DESPUÉS DE StandardScaler:")
print("="*60)
df_standard.describe()

### Interpretación:
- La **media** de cada columna ahora es prácticamente 0
- La **desviación estándar** de cada columna es 1
- Los valores pueden ser negativos (valores por debajo de la media original)

## 1.3 Aplicar MinMaxScaler

**MinMaxScaler** transforma los datos para que estén entre 0 y 1:

Fórmula: $x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$

In [ ]:
# Crear el escalador MinMax
scaler_minmax = MinMaxScaler()

# Aplicar la transformación
datos_escalados_minmax = scaler_minmax.fit_transform(df_limpio[cols_a_escalar])

# Convertir a DataFrame
df_minmax = pd.DataFrame(
    datos_escalados_minmax, 
    columns=[col + '_minmax' for col in cols_a_escalar]
)

print("DATOS DESPUÉS DE MinMaxScaler:")
print("="*60)
df_minmax.describe()

### Interpretación:
- El **mínimo** de cada columna es 0
- El **máximo** de cada columna es 1
- Todos los valores están en el rango [0, 1]

## 1.4 Comparación visual: Antes vs Después

In [ ]:
# Crear figura con 3 gráficos: Original, StandardScaler, MinMaxScaler
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: Datos originales
ax1 = axes[0]
df_limpio[cols_a_escalar].boxplot(ax=ax1)
ax1.set_title('DATOS ORIGINALES\n(Diferentes escalas)', fontweight='bold', fontsize=12)
ax1.set_ylabel('Valor')
ax1.tick_params(axis='x', rotation=45)

# Gráfico 2: StandardScaler
ax2 = axes[1]
df_standard.boxplot(ax=ax2)
ax2.set_title('DESPUÉS DE StandardScaler\n(Media=0, Std=1)', fontweight='bold', fontsize=12)
ax2.set_ylabel('Valor estandarizado')
ax2.tick_params(axis='x', rotation=45)
ax2.axhline(0, color='red', linestyle='--', alpha=0.5, label='Media=0')

# Gráfico 3: MinMaxScaler  
ax3 = axes[2]
df_minmax.boxplot(ax=ax3)
ax3.set_title('DESPUÉS DE MinMaxScaler\n(Valores entre 0 y 1)', fontweight='bold', fontsize=12)
ax3.set_ylabel('Valor normalizado')
ax3.tick_params(axis='x', rotation=45)
ax3.axhline(0, color='blue', linestyle='--', alpha=0.5)
ax3.axhline(1, color='blue', linestyle='--', alpha=0.5)

plt.suptitle('Comparación de Métodos de Escalado', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nOBSERVACIONES:")
print("  - En los datos originales, 'Fare' domina por su escala grande")
print("  - Después del escalado, todas las variables tienen escalas comparables")
print("  - MinMaxScaler acota los valores entre 0 y 1")
print("  - StandardScaler centra en 0 pero puede tener valores fuera de [-1, 1]")

## 1.5 Demostración: Impacto del escalado en KNN

In [ ]:
# Preparar datos para comparar KNN con y sin escalado

# Solo usamos columnas numéricas para este ejemplo
X = df_limpio[cols_a_escalar]
y = df_limpio['Survived']

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 1. KNN SIN escalar
knn_sin_escalar = KNeighborsClassifier(n_neighbors=5)
knn_sin_escalar.fit(X_train, y_train)
acc_sin_escalar = accuracy_score(y_test, knn_sin_escalar.predict(X_test))

# 2. KNN CON StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # fit_transform en train
X_test_scaled = scaler.transform(X_test)        # solo transform en test (IMPORTANTE!)

knn_con_escalar = KNeighborsClassifier(n_neighbors=5)
knn_con_escalar.fit(X_train_scaled, y_train)
acc_con_escalar = accuracy_score(y_test, knn_con_escalar.predict(X_test_scaled))

print("COMPARACIÓN DE PRECISIÓN EN KNN:")
print("="*50)
print(f"Sin escalar:  {acc_sin_escalar:.2%}")
print(f"Con escalar:  {acc_con_escalar:.2%}")
print(f"Mejora:       {(acc_con_escalar - acc_sin_escalar):.2%}")

---

# EJERCICIO 2: Codificación de Variables Categóricas (Básico-Intermedio)

Los algoritmos de ML solo entienden números. Debemos convertir las variables categóricas (Sex, Embarked) a formato numérico.

Técnicas:
1. **Label Encoding**: Asignar un número a cada categoría (0, 1, 2...)
2. **One-Hot Encoding**: Crear una columna binaria por cada categoría

---

## 2.1 Ver las variables categóricas

In [ ]:
print("VARIABLES CATEGÓRICAS EN EL DATASET:")
print("="*50)

for col in columnas_categoricas:
    valores_unicos = df_limpio[col].unique()
    print(f"\n{col}:")
    print(f"  Valores únicos: {valores_unicos}")
    print(f"  Cantidad: {len(valores_unicos)}")
    print(f"  Frecuencias:")
    print(df_limpio[col].value_counts().to_string().replace('\n', '\n    '))

## 2.2 Label Encoding (para variables binarias)

**Label Encoding** es ideal para variables con **2 categorías** (binarias) como `Sex`.

- male → 0
- female → 1

In [ ]:
# Crear copia para no modificar el original
df_encoded = df_limpio.copy()

# Aplicar Label Encoding a 'Sex'
label_encoder = LabelEncoder()
df_encoded['Sex_encoded'] = label_encoder.fit_transform(df_encoded['Sex'])

print("LABEL ENCODING PARA 'Sex':")
print("="*50)
print(f"\nMapeo realizado:")
for i, clase in enumerate(label_encoder.classes_):
    print(f"  {clase} → {i}")

print("\nPrimeras filas con la nueva columna:")
df_encoded[['Sex', 'Sex_encoded']].head(10)

## 2.3 One-Hot Encoding (para variables con múltiples categorías)

**One-Hot Encoding** crea una columna binaria por cada categoría.

Para `Embarked` (C, Q, S):
- Embarked_C: 1 si embarcó en Cherbourg, 0 si no
- Embarked_Q: 1 si embarcó en Queenstown, 0 si no  
- Embarked_S: 1 si embarcó en Southampton, 0 si no

In [ ]:
# Aplicar One-Hot Encoding a 'Embarked' usando pd.get_dummies()
# Esta es la forma más sencilla en pandas

embarked_dummies = pd.get_dummies(df_encoded['Embarked'], prefix='Embarked')

print("ONE-HOT ENCODING PARA 'Embarked':")
print("="*50)
print("\nNuevas columnas creadas:")
print(embarked_dummies.head(10))

In [ ]:
# Añadir las columnas one-hot al DataFrame
df_encoded = pd.concat([df_encoded, embarked_dummies], axis=1)

# Verificar resultado
print("DataFrame con todas las codificaciones:")
df_encoded[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].head(10)

## 2.4 Visualización de la codificación One-Hot

In [ ]:
# Visualizar la matriz One-Hot como un mapa de calor
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Matriz One-Hot (primeras 20 filas)
ax1 = axes[0]
sns.heatmap(
    df_encoded[['Embarked_C', 'Embarked_Q', 'Embarked_S']].head(20),
    cmap='YlGnBu',
    annot=True,
    fmt='d',
    cbar=False,
    ax=ax1
)
ax1.set_title('Matriz One-Hot de Embarked\n(Primeras 20 filas)', fontweight='bold')
ax1.set_ylabel('Pasajero')

# Gráfico 2: Distribución de puertos de embarque
ax2 = axes[1]
colores = ['#2ecc71', '#3498db', '#e74c3c']
df_encoded['Embarked'].value_counts().plot(kind='bar', ax=ax2, color=colores, edgecolor='black')
ax2.set_title('Distribución de Puertos de Embarque', fontweight='bold')
ax2.set_xlabel('Puerto')
ax2.set_ylabel('Número de pasajeros')
ax2.tick_params(axis='x', rotation=0)

# Añadir leyenda
puertos = {'C': 'Cherbourg', 'Q': 'Queenstown', 'S': 'Southampton'}
for i, (puerto, nombre) in enumerate(puertos.items()):
    ax2.text(i, df_encoded['Embarked'].value_counts()[puerto] + 10, 
             nombre, ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---

# EJERCICIO 3: Feature Engineering (Intermedio)

**Feature Engineering** es el arte de crear nuevas columnas a partir de las existentes para mejorar el rendimiento del modelo.

Vamos a crear:
1. `FamilySize`: Tamaño total de la familia
2. `IsAlone`: Si el pasajero viaja solo
3. `AgeGroup`: Categorización de edad
4. `FarePerPerson`: Precio por persona

---

In [ ]:
# Crear copia para feature engineering
df_features = df_limpio.copy()

# 1. FamilySize: SibSp (hermanos/esposos) + Parch (padres/hijos) + 1 (el propio pasajero)
df_features['FamilySize'] = df_features['SibSp'] + df_features['Parch'] + 1

# 2. IsAlone: 1 si FamilySize == 1, 0 si no
df_features['IsAlone'] = (df_features['FamilySize'] == 1).astype(int)

# 3. AgeGroup: Categorizar la edad
def categorizar_edad(edad):
    if edad < 12:
        return 'Niño'
    elif edad < 18:
        return 'Adolescente'
    elif edad < 35:
        return 'Adulto Joven'
    elif edad < 60:
        return 'Adulto'
    else:
        return 'Mayor'

df_features['AgeGroup'] = df_features['Age'].apply(categorizar_edad)

# 4. FarePerPerson: Precio dividido por tamaño de familia
df_features['FarePerPerson'] = df_features['Fare'] / df_features['FamilySize']

print("NUEVAS FEATURES CREADAS:")
print("="*60)
df_features[['Age', 'AgeGroup', 'SibSp', 'Parch', 'FamilySize', 'IsAlone', 'Fare', 'FarePerPerson']].head(10)

In [ ]:
# Visualizar las nuevas features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gráfico 1: Tamaño de familia vs Supervivencia
ax1 = axes[0, 0]
df_features.groupby('FamilySize')['Survived'].mean().plot(kind='bar', ax=ax1, color='steelblue', edgecolor='black')
ax1.set_title('Tasa de Supervivencia por Tamaño de Familia', fontweight='bold')
ax1.set_xlabel('Tamaño de Familia')
ax1.set_ylabel('Tasa de Supervivencia')
ax1.tick_params(axis='x', rotation=0)
ax1.axhline(df_features['Survived'].mean(), color='red', linestyle='--', label='Media global')
ax1.legend()

# Gráfico 2: Solo vs Acompañado
ax2 = axes[0, 1]
supervivencia_solo = df_features.groupby('IsAlone')['Survived'].mean()
bars = ax2.bar(['Acompañado', 'Solo'], supervivencia_solo.values, color=['#27ae60', '#e74c3c'], edgecolor='black')
ax2.set_title('Tasa de Supervivencia: Solo vs Acompañado', fontweight='bold')
ax2.set_ylabel('Tasa de Supervivencia')
for bar, val in zip(bars, supervivencia_solo.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{val:.1%}', ha='center', fontsize=12)

# Gráfico 3: Grupo de edad vs Supervivencia
ax3 = axes[1, 0]
orden_edad = ['Niño', 'Adolescente', 'Adulto Joven', 'Adulto', 'Mayor']
supervivencia_edad = df_features.groupby('AgeGroup')['Survived'].mean().reindex(orden_edad)
supervivencia_edad.plot(kind='bar', ax=ax3, color='coral', edgecolor='black')
ax3.set_title('Tasa de Supervivencia por Grupo de Edad', fontweight='bold')
ax3.set_xlabel('Grupo de Edad')
ax3.set_ylabel('Tasa de Supervivencia')
ax3.tick_params(axis='x', rotation=45)

# Gráfico 4: Distribución de FarePerPerson
ax4 = axes[1, 1]
ax4.hist(df_features['FarePerPerson'], bins=30, color='purple', edgecolor='white', alpha=0.7)
ax4.set_title('Distribución de Precio por Persona', fontweight='bold')
ax4.set_xlabel('Precio por Persona (£)')
ax4.set_ylabel('Frecuencia')
ax4.axvline(df_features['FarePerPerson'].median(), color='red', linestyle='--', label=f"Mediana: £{df_features['FarePerPerson'].median():.2f}")
ax4.legend()

plt.suptitle('Análisis de Features Creadas', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Conclusiones del Feature Engineering:

- Los **niños** tuvieron mayor tasa de supervivencia ("mujeres y niños primero")
- Viajar **acompañado** aumentaba las probabilidades de sobrevivir
- Familias de **2-4 personas** tuvieron mejor supervivencia que los que iban solos o en grupos muy grandes

---

# EJERCICIO 4: Pipeline Completo (Avanzado)

Un **Pipeline** encadena todas las transformaciones en un solo objeto. Ventajas:
1. **Reproducibilidad**: Siempre se aplican los mismos pasos
2. **Prevención de Data Leakage**: El escalador se ajusta solo con datos de train
3. **Código limpio**: Todo en un solo objeto

---

In [ ]:
# Preparar datos para el pipeline
df_pipeline = df_limpio.copy()

# Definir columnas por tipo
columnas_numericas_pipeline = ['Age', 'Fare', 'SibSp', 'Parch']
columnas_categoricas_pipeline = ['Sex', 'Embarked']

# Separar features (X) y target (y)
X = df_pipeline[columnas_numericas_pipeline + columnas_categoricas_pipeline]
y = df_pipeline['Survived']

print("DATOS PARA EL PIPELINE:")
print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nColumnas numéricas: {columnas_numericas_pipeline}")
print(f"Columnas categóricas: {columnas_categoricas_pipeline}")

In [ ]:
# Crear transformadores para cada tipo de columna

# Pipeline para columnas NUMÉRICAS:
# 1. Imputar valores faltantes con la mediana
# 2. Escalar con StandardScaler
transformador_numerico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para columnas CATEGÓRICAS:
# 1. Imputar valores faltantes con 'Desconocido'
# 2. Aplicar One-Hot Encoding
transformador_categorico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

print("Transformadores creados:")
print("  - transformador_numerico: Imputer + StandardScaler")
print("  - transformador_categorico: Imputer + OneHotEncoder")

In [ ]:
# Combinar transformadores con ColumnTransformer
# Esto aplica cada transformador a sus columnas correspondientes

preprocesador = ColumnTransformer(
    transformers=[
        ('num', transformador_numerico, columnas_numericas_pipeline),
        ('cat', transformador_categorico, columnas_categoricas_pipeline)
    ]
)

# Crear pipeline completo: preprocesador + modelo
pipeline_completo = Pipeline(steps=[
    ('preprocesador', preprocesador),
    ('clasificador', KNeighborsClassifier(n_neighbors=5))
])

print("ESTRUCTURA DEL PIPELINE:")
print("="*50)
print(pipeline_completo)

In [ ]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Entrenar el pipeline completo
# fit() automáticamente:
#   1. Ajusta el imputador y escalador SOLO con datos de train
#   2. Entrena el modelo con los datos transformados
pipeline_completo.fit(X_train, y_train)

# Predecir
# predict() automáticamente:
#   1. Transforma X_test usando los parámetros aprendidos de train
#   2. Hace la predicción
y_pred = pipeline_completo.predict(X_test)

# Evaluar
precision = accuracy_score(y_test, y_pred)

print("RESULTADOS DEL PIPELINE:")
print("="*50)
print(f"Precisión en test: {precision:.2%}")
print(f"\nEl pipeline transformó y predijo automáticamente!")

In [ ]:
# Ver cómo quedaron los datos después del preprocesamiento
X_train_transformado = preprocesador.fit_transform(X_train)

# Obtener nombres de las columnas generadas
nombres_columnas = (
    columnas_numericas_pipeline + 
    list(preprocesador.named_transformers_['cat']
         .named_steps['onehot']
         .get_feature_names_out(columnas_categoricas_pipeline))
)

df_transformado = pd.DataFrame(X_train_transformado, columns=nombres_columnas)

print("DATOS DESPUÉS DEL PREPROCESAMIENTO:")
print("="*60)
print(f"\nDimensiones: {df_transformado.shape}")
print(f"\nColumnas generadas: {list(df_transformado.columns)}")
print("\nPrimeras 5 filas:")
df_transformado.head()

---

# Resumen del Ejercicio

En este cuaderno hemos aprendido a:

| Técnica | Herramienta | Uso |
|---------|-------------|-----|
| **Escalado Standard** | `StandardScaler` | Media=0, Std=1. Para algoritmos con gradiente |
| **Escalado MinMax** | `MinMaxScaler` | Valores entre 0 y 1. Para redes neuronales |
| **Label Encoding** | `LabelEncoder` | Variables binarias (Sí/No, Male/Female) |
| **One-Hot Encoding** | `pd.get_dummies()` o `OneHotEncoder` | Variables con múltiples categorías |
| **Feature Engineering** | Código manual | Crear nuevas columnas informativas |
| **Pipeline** | `Pipeline` + `ColumnTransformer` | Automatizar todo el proceso |

---

**¡Felicidades!** Has completado el ejercicio práctico de Transformación de Datos.